# 영어 Word2Vec 만들기

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import re, zipfile
from urllib.request import urlretrieve
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f6a38e91550>)

In [5]:
!more ted*

<?xml version="1.0" encoding="UTF-8"?>
<xml language="en"><file id="1">
  <head>
    <url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_ho
w_to_avoid_them</url>
    <pagesize>72832</pagesize>
    <dtime>Fri Apr 01 00:57:03 CEST 2016</dtime>
    <encoding>UTF-8</encoding>
    <content-type>text/html; charset=utf-8</content-type>
    <keywords>talks, business, creativity, curiosity, goal-setting, innovation, 
motivation, potential, success, work</keywords>
    <speaker>Knut Haanaes</speaker>
    <talkid>2470</talkid>
    <videourl>http://download.ted.com/talks/KnutHaanaes_2015S.mp4</videourl>
    <videopath>talks/KnutHaanaes_2015S.mp4</videopath>
    <date>2015/06/30</date>
    <title>Knut Haanaes: Two reasons companies fail -- and how to avoid them</ti
tle>
    <description>TED Talk Subtitles and Transcript: Is it possible to run a comp
any and reinvent it at the same time? For business strategist Knut Haanaes, the 
ability to innovate after becoming successful is

### 훈련 데이터 전처리하기

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [7]:
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [8]:
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [9]:
send_text = sent_tokenize(content_text)

In [10]:
normalized_text = []
for string in send_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

In [11]:
result = [word_tokenize(sentence) for sentence in normalized_text]

In [12]:
print(f'총 샘플의 개수 : {len(result):,d}')

총 샘플의 개수 : 273,424


In [13]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [14]:
from gensim.models import Word2Vec


In [16]:
%time model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

CPU times: user 1min 2s, sys: 361 ms, total: 1min 3s
Wall time: 36.4 s


In [17]:
model.wv.most_similar("man")

[('woman', 0.8452373147010803),
 ('guy', 0.819441556930542),
 ('lady', 0.7964470386505127),
 ('girl', 0.7699487805366516),
 ('boy', 0.7400234937667847),
 ('soldier', 0.7129745483398438),
 ('gentleman', 0.7098566293716431),
 ('kid', 0.7017723321914673),
 ('friend', 0.6813536286354065),
 ('surgeon', 0.6757211685180664)]

In [18]:
model.wv.most_similar(positive=['man','girl'], negative=['boy'], topn=3)

[('woman', 0.8107026815414429),
 ('lady', 0.7457029819488525),
 ('guy', 0.7404758930206299)]

### 모델 저장하고 로드하기

In [19]:

model.wv.save_word2vec_format('eng_w2v')

In [20]:
!ls -l

total 97836
-rw-r--r-- 1 root root 25644903 Jan 25 01:30 eng_w2v
drwxr-xr-x 1 root root     4096 Jan  7 14:33 sample_data
-rw-r--r-- 1 root root 74533638 Jan 25 01:08 ted_en-20160408.xml


In [23]:
from gensim.models import KeyedVectors

loaded_model = KeyedVectors.load_word2vec_format('eng_w2v')
loaded_model.most_similar("man")

[('woman', 0.8452373147010803),
 ('guy', 0.819441556930542),
 ('lady', 0.7964470386505127),
 ('girl', 0.7699487805366516),
 ('boy', 0.7400234937667847),
 ('soldier', 0.7129745483398438),
 ('gentleman', 0.7098566293716431),
 ('kid', 0.7017723321914673),
 ('friend', 0.6813536286354065),
 ('surgeon', 0.6757211685180664)]